# Part 1
# Web Scraping and Data Storage Application

In [3]:
import csv
import mysql.connector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Insert data into MySQL
def insert_into_mysql(data):
    try:
        connection = mysql.connector.connect(
            host='CSSQL',
            user='mm_team04_02	',
            password='mm_team04_02Pass-',
            database='mm_team04_02'
        )

        cursor = connection.cursor()

        # Assuming your table has columns: product_name, product_price, product_rating
        query = "INSERT INTO best_buy_products (product_name, product_price, product_rating) VALUES (%s, %s, %s)"

        cursor.executemany(query, data)
        connection.commit()

        print(f"{cursor.rowcount} records inserted successfully into MySQL")

    except mysql.connector.Error as err:
        print(f"Error: {err}")

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

In [5]:
def scrape_best_buy(search_term, page_limit=5):
    # Setting up the Chrome webdriver
    driver = webdriver.Chrome()
    base_url = 'https://www.bestbuy.com/'

    try:
        driver.get(base_url)

        # Locating the search bar and enter the search term
        search_bar = driver.find_element(By.ID, 'gh-search-input')
        search_bar.send_keys(search_term)

        # Locating and click the search button
        search_button = driver.find_element(By.CLASS_NAME, 'header-search-button')
        search_button.click()

        current_page = 1

        # Creating a CSV file and writing the header
        with open('best_buy_products.csv', 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ["Product Name", "Product Price", "Product Rating"]
            csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            csv_writer.writeheader()
            # Collectting data to insert into MySQL
            data_to_insert = []  

            while current_page <= page_limit:
                WebDriverWait(driver, 10).until_not(
                    EC.presence_of_element_located((By.ID, 'survey_window'))
                )

                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'sku-item-list'))
                )

                # Getting the page source after the search results have loaded
                page_source = driver.page_source

                # Parsing the HTML content with BeautifulSoup
                soup = BeautifulSoup(page_source, 'html.parser')

                # Finding the parent <ol> element that contains the <li> elements
                ol_element = soup.find('ol', class_='sku-item-list')  # Update with the actual class name

                # Checking if the <ol> element is found
                if ol_element:
                    li_elements = ol_element.find_all('li', class_='sku-item')

                    for product_element in li_elements:
                        # Extracting product information 
                        product_name_element = product_element.find('h4', class_='sku-title')
                        product_price_element = product_element.find('div', class_='priceView-customer-price')
                        product_rating_element = product_element.find('div', class_='c-ratings-reviews')

                        # Checking if elements are found before accessing 'text' attribute
                        if product_name_element and product_price_element:
                            # Extracting product name from the anchor tag within h4
                            product_name = product_name_element.find('a').text.strip()

                            # Extracting price information directly from the span element
                            price_span = product_price_element.find('span', aria_hidden='true') or product_price_element.find('span')
                            product_price = price_span.text.strip() if price_span else "Price information not available"

                            # Extracting rating information
                            rating_text = product_rating_element.find('p', class_='visually-hidden').text.strip()
                            rating = rating_text.split(' ')[1] if rating_text else "Rating information not available"

                            # Writing the data to the CSV file
                            csv_writer.writerow({
                                "Product Name": product_name,
                                "Product Price": product_price,
                                "Product Rating": rating
                            })

                            # Appending data to the list
                            data_to_insert.append((product_name, product_price, rating))

                next_page_button = driver.find_element(By.CLASS_NAME, 'sku-list-page-next')
                if 'disabled' not in next_page_button.get_attribute('class'):
                    next_page_button.click()
                    current_page += 1
                else:
                    break

            # After scraping is complete, inserting data into MySQL
            insert_into_mysql(data_to_insert)

    except Exception as e:
        print(f"Error: {e}")

    finally:
        driver.quit()

# Replace 'laptop' with the  product name you want to search for
# Set the page limit to the desired number (e.g., page_limit=5)
scrape_best_buy('Iphone', page_limit=5)

89 records inserted successfully into MySQL


In [7]:
scrape_best_buy('Samsung', page_limit=5)

89 records inserted successfully into MySQL


# Part 2:
# Moving Data from Relational Database to MongoDB

In [1]:
import mysql.connector
from pymongo import MongoClient

def move_data_mysql_to_mongodb():
    try:
        # Connecting to MySQL database
        mysql_conn = mysql.connector.connect(
            host="CSSQL",
            user="mm_team04_02",
            password="mm_team04_02Pass-",
            database="mm_team04_02"
        )
        mysql_cursor = mysql_conn.cursor()

        # Connecting to MongoDB database
        mongo_client = MongoClient("mongodb://localhost:27017/")
        mongo_db = mongo_client["EliteWork"]  # Use "EliteWork" as the database name
        mongo_collection = mongo_db["Customer"]

        # Moveing data from MySQL to MongoDB for customer table
        mysql_cursor.execute("SELECT * FROM customer")
        customers = mysql_cursor.fetchall()
        for customer in customers:
            customer_data = {
                "_id": customer[0],  # Assuming CUSTOMER_ID is unique
                "name": customer[1],
                "email": customer[2],
                "phone_number": customer[3],
                "store_id": customer[4]
            }
            mongo_collection.insert_one(customer_data)

        # Moveing data from MySQL to MongoDB for sales table
        mongo_collection = mongo_db["Sales"]
        mysql_cursor.execute("SELECT * FROM sales")
        sales = mysql_cursor.fetchall()
        for sale in sales:
            sale_data = {
                "_id": {"store_id": sale[4], "invoice_num": sale[0]},  # Composite primary key
                "sales_date": sale[1],
                "quantity": sale[2],
                "total_amount": float(sale[3])  # Convert to float
            }
            mongo_collection.insert_one(sale_data)

        # Moving data from MySQL to MongoDB for store table
        mongo_collection = mongo_db["Store"]
        mysql_cursor.execute("SELECT * FROM store")
        stores = mysql_cursor.fetchall()
        for store in stores:
            store_data = {
                "_id": store[0],  # Assuming STORE_ID is unique
                "zip_code": store[1],
                "state": store[2]
            }
            mongo_collection.insert_one(store_data)

        print("Data moved successfully from MySQL to MongoDB.")

    except mysql.connector.Error as mysql_err:
        print(f"MySQL Error: {mysql_err}")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        # Closing MySQL connection
        if mysql_conn.is_connected():
            mysql_cursor.close()
            mysql_conn.close()
        # Closing MongoDB connection
        mongo_client.close()

# Calling the function to move data from MySQL to MongoDB
move_data_mysql_to_mongodb()


Data moved successfully from MySQL to MongoDB.


# Part 3:
# Moving Data from CSSQL Database to Amazon RDS Database

In [4]:
import mysql.connector
 
# Defining MySQL connection details for the local server
local_mysql_connection = mysql.connector.connect(
    host="CSSQL",
    user="mm_team04_02",
    password="mm_team04_02Pass-",
    database="mm_team04_02"
)
 
# Defining MySQL connection details for the RDS instance
rds_mysql_connection = mysql.connector.connect(
    host="elitework.cpm2mgw221ml.us-east-1.rds.amazonaws.com",
    user="admin",
    password="Bhusal9891t",
    database="mm_team04_02"
)
 
# Creating cursors for both connections
local_mysql_cursor = local_mysql_connection.cursor()
rds_mysql_cursor = rds_mysql_connection.cursor()
 
# Defining the list of tables to copy
tables_to_copy = ['supplier','store']
 
# Iterating over each table and copy its structure and data
for table_name in tables_to_copy:
    try:
        # Getting the CREATE TABLE query for the table
        local_mysql_cursor.execute(f"SHOW CREATE TABLE {table_name}")
        create_table_query = local_mysql_cursor.fetchone()[1]
        
        # Executing the CREATE TABLE query on the RDS instance
        rds_mysql_cursor.execute(create_table_query)
        
        # Fetching all rows from the table on the local server
        local_mysql_cursor.execute(f"SELECT * FROM {table_name}")
        rows = local_mysql_cursor.fetchall()
        
        # Insertting each row into the corresponding table on the RDS instance
        for row in rows:
            insert_query = f"INSERT INTO {table_name} VALUES ({', '.join(['%s'] * len(row))})"
            rds_mysql_cursor.execute(insert_query, row)
    except mysql.connector.Error as err:
        print(f"An error occurred while copying table {table_name}: {err}")
 
# Committing changes and close connections
rds_mysql_connection.commit()
local_mysql_cursor.close()
rds_mysql_cursor.close()
local_mysql_connection.close()
rds_mysql_connection.close()

print("Code ran successfully!!")

Code ran successfully!!
